<a href="https://colab.research.google.com/github/k1151msarandega/QuCode-21-Days-of-Quantum-Challenge-Diary/blob/main/Day13_Quantum_Computing_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Day 13 — Quantum Computing Models

> _QuCode 21 Days of Quantum Challenge — Learning notebook_
>
> **Date:** 2025-09-03  
> **Author:** Kudzai Musarandega  
> **Tags:** quantum, learning, challenge, day-13
>
> **Learning objectives**
> - Contrast the **circuit model**, **adiabatic quantum computation (AQC)**, and **measurement-based quantum computation (MBQC)**.
> - Implement Rau's MBQC primitive that realises a single-qubit unitary using only measurements and classical feed-forward.
> - Simulate a toy two-level **adiabatic evolution**, study fidelity vs total time, and relate it to the **minimum spectral gap**.
> - Build a small **clause Hamiltonian** for SAT and inspect its spectrum.
>
> **Key takeaways (summary-first)**
> - Circuit universality with small gate sets (e.g., {H, T, CNOT}).
> - MBQC can replace explicit unitaries with measurements; expected two trials to succeed for the single-qubit primitive.
> - Adiabatic run time is limited by the **inverse square of the minimum gap** along the path.
> - SAT-to-Hamiltonian maps clauses to energy penalties; ground states encode satisfying assignments.

## Resources
- **Official/Assigned:**
    - [Jochen Rau: Measurement-based computation](https://www.youtube.com/watch?v=7tf3I8gYkOs)
    - [Sandro Mareco: ADIABATIC QUANTUM COMPUTATION](https://youtu.be/vFpNNrt-baE?si=4MpI3nPI8RodyaTq)
- **Extra reading:**
- **Original notes:**


In [ ]:
# %% [markdown]
# ### Environment setup (Colab)
# If you are running on Colab for the first time today, uncomment to install.
# This cell intentionally avoids heavy installs by default.
#
# !pip -q install qiskit pennylane matplotlib numpy

import sys, platform, math, json, numpy as np

print("Python:", sys.version.split()[0])
print("Platform:", platform.platform())
np.random.seed(42)


## 1. Concepts in brief

### Circuit model
- Build poly-size unitaries from a universal gate set. Example minimal set: single-qubit {H, T} and two-qubit CNOT.
- Measure in the computational basis to obtain classical output.

### Measurement-based QC (MBQC)
- Prepare an entangled resource (e.g., a cluster state), then drive computation by single-site measurements with adaptive bases.
- **Rau’s proof-of-principle operator**
  $
  B=\tfrac12\big[(\sigma_x - i\sigma_y)\!\otimes\! U + (\sigma_x + i\sigma_y)\!\otimes\! U^\dagger\big],
  $
  with $B^\dagger=B$, $B^2=\mathbb{I}$, spectrum $\{\pm1\}$.
  Crucially,
  $
  B\big(|0\rangle\!\otimes\!|\psi\rangle\big)=|1\rangle\!\otimes\!U|\psi\rangle.
  $
  Follow with a Z-measurement on the ancilla and classical feed-forward; expected number of tries is 2.

### Adiabatic quantum computation (AQC)
- Evolve under $H(s)=(1-s)H_0+sH_f$, $s\in[0,1]$, starting in the ground state of $H_0$.
- **Adiabatic condition (scaling form):**
  $
  T \gtrsim \frac{1}{\varepsilon}\max_s \frac{\|\partial_s H(s)\|}{g(s)^2},
  $
  where $g(s)$ is the instantaneous spectral gap.
- **SAT encoding:** sum clause penalty terms $H_f=\sum_j H_{C_j}$ where $H_{C_j}$ assigns energy 1 only to unsatisfied assignments. Ground energy 0 iff the instance is satisfiable.


In [2]:
import numpy as np
import numpy.linalg as la
import math
import matplotlib.pyplot as plt

np.set_printoptions(precision=4, suppress=True)

I2 = np.eye(2, dtype=complex)
X  = np.array([[0,1],[1,0]], dtype=complex)
Y  = np.array([[0,-1j],[1j,0]], dtype=complex)
Z  = np.array([[1,0],[0,-1]], dtype=complex)

def kron(*ops):
    M = np.array([[1+0j]])
    for A in ops:
        M = np.kron(M, A)
    return M

def dagger(M):
    return M.conj().T

def ket(v):
    v = np.array(v, dtype=complex).reshape(-1,1)
    n = la.norm(v)
    return v/n if n>0 else v

def fidelity(psi, phi):
    return float(abs(dagger(psi) @ phi)**2)

def projector(v):
    v = ket(v)
    return v @ dagger(v)

def pretty(psi, tol=1e-9):
    psi = psi.flatten()
    n = int(round(math.log2(psi.size)))
    terms = []
    for i, amp in enumerate(psi):
        if abs(amp) > tol:
            terms.append(f"{amp:.3g}|{i:0{n}b}>")
    return " + ".join(terms) if terms else "0"


## 2a. Worked examples (MBQC operator B demo)

In [3]:
# Worked examples (MBQC operator B demo)

import numpy as np

def dagger(A):
    return A.conj().T

# Pauli matrices
I2 = np.eye(2, dtype=complex)
sx = np.array([[0,1],[1,0]], dtype=complex)
sy = np.array([[0,-1j],[1j,0]], dtype=complex)
sz = np.array([[1,0],[0,-1]], dtype=complex)

# Basis kets
ket0 = np.array([[1.0],[0.0]], dtype=complex)
ket1 = np.array([[0.0],[1.0]], dtype=complex)

def kron(*ops):
    M = np.array([[1.0+0j]])
    for A in ops:
        M = np.kron(M, A)
    return M

def random_unitary_2():
    # Haar via QR
    X = (np.random.randn(2,2) + 1j*np.random.randn(2,2))/np.sqrt(2)
    Q,R = np.linalg.qr(X)
    d = np.diag(R)
    ph = d/np.abs(d)
    return Q @ np.diag(ph)

def normalise(v):
    n = np.linalg.norm(v)
    return v/n if n>0 else v

# Build B for a given 1-qubit unitary U
def build_B(U):
    term1 = kron((sx - 1j*sy)/2, U)
    term2 = kron((sx + 1j*sy)/2, dagger(U))
    return term1 + term2

# Demo
np.random.seed(7)
U = random_unitary_2()
B = build_B(U)

# Checks: Hermitian and idempotent square to identity
hermitian_ok = np.allclose(B, dagger(B), atol=1e-10)
square_ok = np.allclose(B @ B, np.eye(4, dtype=complex), atol=1e-10)

# Action on |0>⊗|ψ> equals |1>⊗U|ψ>
psi = normalise(np.random.randn(2,1)+1j*np.random.randn(2,1))
lhs = B @ kron(ket0, psi)
rhs = kron(ket1, U @ psi)
action_ok = np.allclose(lhs, rhs, atol=1e-10)

print(f"Hermitian: {hermitian_ok},  B^2=I: {square_ok},  Correct action: {action_ok}")


Hermitian: True,  B^2=I: True,  Correct action: True


## 2b. Worked examples (AQC toy gap)

In [4]:
# Worked examples (AQC toy gap)

import numpy as np

# H(s) = (1-s)*Δ σx + s*δ σz for a single qubit; compute instantaneous gap and its minimum
def H_single(s, Delta=1.0, delta=0.3):
    return (1-s)*Delta*sx + s*delta*sz

def gap(s, Delta=1.0, delta=0.3):
    # For 2x2 Hermitian, eigenvalues ±||h||; gap = 2*norm of Bloch vector
    hx, hz = (1-s)*Delta, s*delta
    return 2.0*np.sqrt(hx*hx + hz*hz)

def min_gap(Delta=1.0, delta=0.3, grid=10001):
    ss = np.linspace(0,1,grid)
    gs = np.array([gap(s, Delta, delta) for s in ss])
    idx = np.argmin(gs)
    return ss[idx], gs[idx]

for Delta, delta in [(1.0, 0.3), (1.0, 1.0), (2.0, 0.2)]:
    s_star, gmin = min_gap(Delta, delta)
    print(f"Δ={Delta:.2f}, δ={delta:.2f} -> s*≈{s_star:.4f}, g_min≈{gmin:.6f}, suggested T ∝ 1/g_min^2≈{(1/gmin**2):.2f}")


Δ=1.00, δ=0.30 -> s*≈0.9174, g_min≈0.574696, suggested T ∝ 1/g_min^2≈3.03
Δ=1.00, δ=1.00 -> s*≈0.5000, g_min≈1.414214, suggested T ∝ 1/g_min^2≈0.50
Δ=2.00, δ=0.20 -> s*≈0.9901, g_min≈0.398015, suggested T ∝ 1/g_min^2≈6.31


## 3. Key equations and statements

- **Circuit universality:** {H, T, CNOT} generates all poly-size circuits.
- **MBQC steering operator**
  $
  B=\tfrac12\big[(\sigma_x - i\sigma_y)\otimes U + (\sigma_x + i\sigma_y)\otimes U^\dagger\big],\quad
  B^\dagger=B,\; B^2=\mathbb{I},\; \mathrm{spec}(B)=\{\pm1\}.
  $
  Post-measurement projector: $\Pi_\pm=\tfrac12(\mathbb{I}\pm B)$.
- **Adiabatic path:** $H(s)=(1-s)H_0+sH_f$.
- **Gap condition (scaling):** for error $\varepsilon$,
  $
  T \gtrsim \frac{1}{\varepsilon}\max_s \frac{\|\partial_s H(s)\|}{g(s)^2}.
  $
- **Clause penalty example (3-SAT):** for clause $x_1\lor x_2\lor x_3$, $H_C$ is diagonal with 1 only on $|000\rangle\langle 000|$, 0 elsewhere.

## 4. Try it yourself

1. **MBQC algebra:** starting from the definition of $B$, verify symbolically (or numerically) that
   $B(|0\rangle\!\otimes\!|\psi\rangle)=|1\rangle\!\otimes\!U|\psi\rangle$ and $\mathbb E[B]=0$ on $|0\rangle\!\otimes\!|\psi\rangle$.

2. **MBQC universality sketch:** outline a measurement pattern to realise CNOT using an ancilla and suitable two-qubit observables.

3. **AQC gap tuning:** modify $\Delta,\delta$ in the toy model and map $g_{\min}$ versus parameters. Discuss how this affects the required $T$.

4. **SAT Hamiltonian mini-build:** for two clauses on three qubits, construct $H_f$, list all eigenvalues, and identify the ground-space assignments.


## 5. Reflection

- Which model feels most intuitive to you today, and why?  
- Where do you see the clearest trade-off between theoretical elegance and practicality?  
- One sentence to future you about MBQC and AQC to remember later.

---
### Links
- **Open in Colab (from GitHub):** replace `YOUR_GITHUB_USERNAME/qucode-21days`
  - `https://colab.research.google.com/github/YOUR_GITHUB_USERNAME/qucode-21days/blob/main/Day13_Grover’s_Search.ipynb.ipynb`
- **Report an issue / suggest a fix:** link to your repo issues page
